# SIPP Data Processing

In this notebook, I process the data from the US Census Bureau Survey of Income and Program Participation (SIPP) from years 2018 - 2021 and export the relevant data to SIPP_data.csv.


In [1]:
import pandas as pd
import numpy as np

In [142]:
np.nan*2

nan

In [146]:
def calculate_annual_wage(row):
    """
    Calculate the annual wage for person given reported wage type.
    """
    if row['TJB1_ANNSAL1'] > 0:
        return row['TJB1_ANNSAL1']
    elif row['TJB1_HOURLY1'] > 0:
        return row['TJB1_HOURLY1']*row['TJB1_JOBHRS1']*52
    elif row['TJB1_WKSUM1'] > 0:
        return row['TJB1_WKSUM1']*52
    elif row['TJB1_SMTHLY1'] > 0:
        return row['TJB1_SMTHLY1']*24
    elif row['TJB1_MSUM'] > 0:
        return row['TJB1_MSUM']*12
    elif row['TJB1_BWKLY1'] > 0:
        return row['TJB1_BWKLY1']*26
    else:
        return 0

In [147]:
def prep_data(data_file, rd_schema, rw_schema, year):
    """ 
    Process SIPP survey data.
    First read in schema to get data-type information for each varaible (rd_schema). 
    Then read in relevant columns from census data file (data_file).
    Extract age of youngest child by finding max of TCBYR columns.
    Keep only columns corresponding to parents with a child under 15.
    Calculate Annual Wage
    Add survey year column.
    """
    rd_schema = pd.read_json(rd_schema)
    rw_schema = pd.read_json(rw_schema)
    rd_schema['dtype'] = ['Int64' if x== 'integer' \
                         else 'object' if x == 'string' \
                         else 'Float64' if x == 'float' \
                         else 'ERROR' \
                         for x in rd_schema['dtype']]
    rw_schema['dtype'] = ['Int64' if x== 'integer' \
                         else 'object' if x == 'string' \
                         else 'Float64' if x == 'float' \
                         else 'ERROR' \
                         for x in rw_schema['dtype']]
    data_df = pd.read_csv(data_file, \
                          names = rd_schema['name'], \
                          dtype = dict([(i,v) for i,v in zip(rd_schema['name'], rd_schema['dtype'])]), \
                          sep = '|', \
                          header = 0, \
                          usecols = ['SSUID','PNUM','SPANEL','TST_INTV','RREGION_INTV',\
                                     'TMETRO_INTV','TAGE', 'ESEX','ERACE','EORIGIN','EEDUC','TCBYR_1','TCBYR_2',\
                                     'TCBYR_3','TCBYR_4','TCBYR_5','TCBYR_6','EDAYCARE', 'EDAYHS', 'EFAM', \
                                     'EGRAN', 'EHEADST','EJB1_AWOP1','EJB1_AWOPRE1','EJB1_AWOPSM1', \
                                     'EJB1_PTRESN1','ELIST','ENJ_NOWRK6','ENREL','ENUR','ENURHS', \
                                     'EOTHR','EPAR','EPAY','EPAYHELP','EPROG','ESELF','ESIB15', \
                                     'ETIMELOST','ETIMELOST_TP','EWORKMORE','EJB1_TYPPAY1','TJB1_ANNSAL1','TJB1_HOURLY1',\
                                     'TJB1_WKSUM1','TPAYWK','TJB1_MSUM','TJB1_BWKLY1','TJB1_SMTHLY1','TJB1_JOBHRS1']
                         )
    data_df['MAX_TCBYR'] = data_df[['TCBYR_1','TCBYR_2','TCBYR_3','TCBYR_4','TCBYR_5','TCBYR_6']].max(axis=1)
    data_df = data_df[year-data_df['MAX_TCBYR']<=14]
    data_df['YEAR'] = year
    data_df['CHILDAGE'] = data_df['YEAR'] - data_df['MAX_TCBYR']
    data_df = data_df.fillna(0)
    data_df['YEARLYWAGE'] = data_df.apply(lambda row: calculate_annual_wage(row), axis=1)
    data_df = data_df.drop_duplicates()
    
    return data_df

In [148]:
SIPP_2018 = prep_data('pu2018.csv', 'pu2018_schema.json', 'rw2018_schema.json', 2018)
SIPP_2019 = prep_data('pu2019.csv', 'pu2019_schema.json', 'rw2019_schema.json', 2019)
SIPP_2020 = prep_data('pu2020.csv', 'pu2020_schema.json', 'rw2020_schema.json', 2020)
SIPP_2021 = prep_data('pu2021.csv', 'pu2021_schema.json', 'rw2021_schema.json', 2021)

In [149]:
dfs = [SIPP_2018, SIPP_2019, SIPP_2020, SIP_2021]

In [150]:
SIPP_data = pd.concat(dfs, ignore_index=True)

In [151]:
SIPP_data.to_csv('SIPP_data.csv')

In [154]:
SIPP_data.head()

,SSUID,SPANEL,PNUM,ESEX,EORIGIN,ERACE,EEDUC,EPAR,ESELF,EGRAN,ESIB15,EOTHR,EFAM,ENREL,EDAYCARE,EDAYHS,EHEADST,EPROG,EPAY,EPAYHELP,ELIST,EWORKMORE,ETIMELOST,ETIMELOST_TP,EJB1_TYPPAY1,EJB1_PTRESN1,ENJ_NOWRK6,EJB1_AWOP1,EJB1_AWOPSM1,EJB1_AWOPRE1,RREGION_INTV,TMETRO_INTV,TST_INTV,TJB1_ANNSAL1,TJB1_HOURLY1,TJB1_BWKLY1,TJB1_SMTHLY1,TJB1_JOBHRS1,TPAYWK,TAGE,TCBYR_1,TCBYR_2,TCBYR_3,TCBYR_4,TCBYR_5,TCBYR_6,TJB1_WKSUM1,TJB1_MSUM,ENUR,ENURHS,MAX_TCBYR,YEAR,CHILDAGE,YEARLYWAGE,MONTHCODE
0,00011413607018,2018,101,1,1,1,38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0,0,2,1,20,0,0.0,0,0,40,0,33,2010,2015,0,0,0,0,400.0,1771,0,0,2015.0,2018,3.0,20800.0,<NA>
1,00011413607018,2018,101,1,1,1,38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0,0,2,1,20,0,0.0,0,0,40,0,33,2010,2015,0,0,0,0,400.0,1600,0,0,2015.0,2018,3.0,20800.0,<NA>
2,00011413607018,2018,101,1,1,1,38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0,0,2,1,20,0,0.0,0,0,40,0,33,2010,2015,0,0,0,0,400.0,1714,0,0,2015.0,2018,3.0,20800.0,<NA>
3,00028504904018,2018,101,1,1,1,38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,2,20,0,0.0,0,0,0,0,44,1996,1998,2005,0,0,0,0.0,0,0,0,2005.0,2018,13.0,0.0,<NA>
4,00028504904018,2018,102,2,1,1,38,1,1,2,0,2,2,2,2,0,0,2,2,2,2,2,0,0,1,0,0,2,0,0,2,2,20,0,0.0,640,0,40,0,41,1996,1998,2005,0,0,0,320.0,1417,0,0,2005.0,2018,13.0,16640.0,<NA>
